In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from globalfunctions import *
%matplotlib inline

tickers = ['SPY','EFA','IEF','GLD']

In [2]:
# format epoch time to datetime
def convertTime(dtEpoch):
    formattedDate = datetime.datetime.fromtimestamp(dtEpoch).strftime('%Y-%m-%d')
    return formattedDate

# Get Historical Data
def GetModalData(tickers = []):
    lst_data = {}
    key =  getData("SELECT [keyValue] FROM [fattailinvestorweb].[dbo].[tblAPIKeys] where pkAPIID = 1")
    for t in tickers:
        # Get Price data
        priceHistory = get_price_history(key['keyValue'][0],symbol=t, period=1, periodType='year', frequencyType='weekly')
        # focus on dict item with price history
        data = priceHistory['candles']
        # loop through dictionary to get close price data
        lst_price = []
        lst_date = []
        for d in data:
            close = (d['close']+d['high'] + d['low'])/3
            date = convertTime((d['datetime']/1000)+(86400*5))
            lst_price.append(close)
            lst_date.append(date)
        if 'Date' not in lst_data:
            lst_data['Date'] = lst_date
        lst_data[t] = lst_price
        

    return lst_data

    

In [3]:
lst = GetModalData(tickers)


In [4]:
print(lst)

{'Date': ['2020-02-07', '2020-02-14', '2020-02-21', '2020-02-28', '2020-03-06', '2020-03-13', '2020-03-20', '2020-03-27', '2020-04-03', '2020-04-10', '2020-04-17', '2020-04-24', '2020-05-01', '2020-05-08', '2020-05-15', '2020-05-22', '2020-05-29', '2020-06-05', '2020-06-12', '2020-06-19', '2020-06-26', '2020-07-03', '2020-07-10', '2020-07-17', '2020-07-24', '2020-07-31', '2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28', '2020-09-04', '2020-09-11', '2020-09-18', '2020-09-25', '2020-10-02', '2020-10-09', '2020-10-16', '2020-10-23', '2020-10-30', '2020-11-06', '2020-11-13', '2020-11-20', '2020-11-27', '2020-12-04', '2020-12-11', '2020-12-18', '2020-12-25', '2021-01-01', '2021-01-08', '2021-01-15', '2021-01-22', '2021-01-29'], 'SPY': [329.87, 335.6366666666667, 335.0466666666666, 305.12076666666667, 300.51, 268.50666666666666, 237.90666666666667, 244.82666666666668, 251.80666666666664, 269.18993333333333, 281.78333333333336, 280.5937333333333, 286.3966666666667, 288.17333333333335, 2

In [5]:
df = pd.DataFrame(lst,columns=['Date','SPY','EFA','IEF','GLD'])
df.tail()

,Date,SPY,EFA,IEF,GLD
47,2021-01-01,373.123333,73.250000,119.829000,177.443333
48,2021-01-08,375.856667,74.836667,118.810000,176.010000
49,2021-01-15,376.843333,74.705000,118.188333,172.188333
50,2021-01-22,381.526667,75.115000,118.468333,173.776667
51,2021-01-29,382.700000,74.858333,118.930000,173.923333


In [6]:
dfDic = {}
for t in tickers:
    colIndex = df.columns.get_loc(t)
    dfDic[t] = colIndex
print(dfDic)

{'SPY': 1, 'EFA': 2, 'IEF': 3, 'GLD': 4}


In [7]:
def CalculateROC(dataFrame, dfDic,N):
    df = dataFrame.copy()
    for key in dfDic:
        df[key+"_ROC"] = np.NaN
        rocIndex = df.columns.get_loc(key+"_ROC")
        for i,j in df.iterrows():
            if i>=N-1:
                df.iloc[i,rocIndex] = (df.iloc[i,dfDic[key]]/df.iloc[i-N,dfDic[key]])-1
    return df
                    
        

In [8]:
df = CalculateROC(df,dfDic,43)
df.tail()

,Date,SPY,EFA,IEF,GLD,SPY_ROC,EFA_ROC,IEF_ROC,GLD_ROC
47,2021-01-01,373.123333,73.250000,119.829000,177.443333,0.241634,0.172813,0.002250,0.142731
48,2021-01-08,375.856667,74.836667,118.810000,176.010000,0.399804,0.390284,0.003576,0.188641
49,2021-01-15,376.843333,74.705000,118.188333,172.188333,0.583997,0.570808,-0.001675,0.222842
50,2021-01-22,381.526667,75.115000,118.468333,173.776667,0.558354,0.467663,-0.017214,0.158022
51,2021-01-29,382.700000,74.858333,118.930000,173.923333,0.519817,0.444630,-0.023590,0.150312


In [9]:
dfSummary = {}
for t in tickers:
    colTitle = t+'_ROC'
    roc = df[colTitle][51]
    dfSummary[colTitle] = roc
print(dfSummary)

{'SPY_ROC': 0.519816790659501, 'EFA_ROC': 0.44463027885883366, 'IEF_ROC': -0.023589940067321735, 'GLD_ROC': 0.15031195573094625}
